In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True,
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor()
)
print (train_data)
print (test_data)
num_epochs = 20
batch_size = 128

learning_rate = 0.001
from torch.utils.data import DataLoader
loaders = {
    'train' : torch.utils.data.DataLoader(train_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1),

    'test'  : torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=1),
}
loaders
### Encoder
#creation using

#Conv2D, MaxPooling2D,Conv2D, MaxPooling2D,Conv2D, MaxPooling2D
###

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        return x
###Decoder creation using:

#Conv2D, UpSampling2D,Conv2D, UpSampling2D,Conv2D, UpSampling2D,Final Conv2D to get image

class Decoder (nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=3,stride=2, padding=1, output_padding=1)
        self.conv2 = nn.ConvTranspose2d(in_channels=8, out_channels=8, kernel_size=3,stride=2, padding=1, output_padding=1)
        self.conv3 = nn.ConvTranspose2d(in_channels=8, out_channels=16, kernel_size=3,stride=2, padding=0, output_padding=1)
        self.conv4 = nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=3,padding=0)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        self.batch_norm = nn.BatchNorm2d(1)

    def forward(self, x):

        x = self.conv1(x)
        x = nn.ReLU()(x)
       # x = self.upsample(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
       # x = self.upsample(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
      #  x = self.upsample(x)
        x = self.conv4(x)

        x = nn.Sigmoid()(x)
        #x = self.batch_norm(x)


        return x

### Autoencoder creation using:
# Encoder, Decoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Move your model to the device (GPU if available, else CPU)
model = Autoencoder().to(device)




loss_func = nn.MSELoss()
 #loss_func = nn.CrossEntropyLoss()
from torch.autograd import Variable

from torch import optim
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
 ### Training


100%|██████████| 9912422/9912422 [00:00<00:00, 112375898.71it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 20341846.15it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27703458.57it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3361067.18it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


"\nfor epoch in range(num_epochs):\n    for i, (images, _) in enumerate(loaders['train']):\n        images = images.to(device)  # Move data to the GPU if available\n        optimizer.zero_grad()\n        outputs = model(images)\n        loss = loss_func(outputs, images)\n        loss.backward()\n        optimizer.step()\n        if (i+1) % 100 == 0:\n            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'\n                  % (epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))\n\n# Testing\nmodel.eval()\nfor images, _ in loaders['test']:\n    images = images.to(device)  # Move data to the GPU if available\n    outputs = model(images)\n    break\n\n# Move outputs and images back to CPU for visualization (if necessary)\noutputs = outputs.cpu().data.numpy()\nimages = images.cpu().data.numpy()\n\nprint (outputs.shape)\nprint (images.shape)\nn = 10\nplt.figure(figsize=(20, 4))\nfor i in range(n):\n    ax = plt.subplot(2, n, i+1)\n    plt.imshow(images[i][0], cmap='gray'

In [ ]:
list_loss_train = []
list_accuracy_train = []
list_loss_test = []
list_accuracy_test = []

y_pred =[]
y_true =[]
from torch.autograd import Variable
seuil=0.5
for epoch in range(num_epochs):
    model.train()
    sous_list_loss_train = []
    sous_valeur_accuracy_train = 0
    #for i, (inputs_train, labels_train ) in enumerate(loaders['train']):
    for i, (images, labels_train) in enumerate(loaders['train']):
        images = images.to(device)  # Move data to the GPU if available
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, images)
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                  % (epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))


        sous_list_loss_train.append(loss.item())
        sous_valeur_accuracy_train =(outputs > 0.5).float().eq(images > 0.5).sum().item()
    accuracy = (sous_valeur_accuracy_train / images.numel()) * 100
    print(accuracy )
    #total_pixels = images.view(images.size(0), -1).size(1)
    #list_accuracy_train.append(sous_valeur_accuracy_train /( len(loaders['train']) * total_pixels))
    list_accuracy_train.append(accuracy)
    list_loss_train.append(np.mean(sous_list_loss_train))



    model.eval()
    with torch.no_grad():
        sous_list_loss_test = []
        sous_valeur_accuracy_test = 0
        for images, labels_test in loaders['test']:
            images = images.to(device)  # Move data to the GPU if available
            outputs = model(images)


            loss = loss_func(outputs,  images)
            sous_list_loss_test.append(loss.item())
            sous_valeur_accuracy_test =  (outputs > 0.5).float().eq(images > 0.5).sum().item()
        accuracy_test = (sous_valeur_accuracy_test / images.numel()) * 100
        print(accuracy_test)
        list_accuracy_test.append(accuracy_test)

        list_loss_test.append(np.mean(sous_list_loss_test))


Epoch [1/20], Iter [100/468] Loss: 0.1237


In [ ]:
import numpy as np
outputs = outputs.cpu().data.numpy()
images = images.cpu().data.numpy()

print (outputs.shape)
print (images.shape)
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(images[i][0], cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(outputs[i][0], cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list_accuracy_train, label='Training accuracy')
#plt.show()
plt.plot(list_accuracy_test, label='Test accuracy')
plt.legend()

    # Show the figure
plt.show()
plt.plot(list_loss_train, label='Training loss')
plt.plot(list_loss_test , label='Test loss')
plt.legend()
    # Show the figure
plt.show()